In [1]:
from tqdm import tqdm
import torch
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
OUTPUT_DIR = "./output/"

In [7]:
cube_img = cv2.imread("output/cube_map.png")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
def create_3dmap_from_size_torch(img_w, img_h, device):
    # np.linspaceをtorch.linspaceに置き換え
    h = torch.linspace(-np.pi/2, np.pi/2, img_h, device=device)
    w = torch.linspace(-np.pi, np.pi, img_w, device=device)
    
    # オフセットの追加
    h += (np.pi/2) / img_h
    w += np.pi / img_w
    
    # np.meshgridをtorch.meshgridに置き換え
    theta, phi = torch.meshgrid(w, h, indexing="ij")
    
    # 3D座標の計算
    x = torch.cos(phi) * torch.cos(theta)
    y = torch.cos(phi) * torch.sin(theta)
    z = torch.sin(phi)
    
    return x, y, z

In [35]:
def padding_cube(img, device):
    # Convert input numpy array to PyTorch tensor
    img_tensor = torch.tensor(img).to(device).permute(0, 1, 2)
    
    h, w, c = img.shape
    cw = w // 4
    print(h, w, c)
    
    # Initialize canvas tensor
    canvas = torch.zeros((h+4, w+4, c), dtype=img_tensor.dtype, device=device)
    canvas[2:-2, 2:-2,:] = img_tensor
    
    # up    
    canvas[0:2, cw+2:2*cw+2,:] = torch.rot90(img_tensor[cw:cw+2, 3*cw:,:], 2, [0,1])
    # bottom
    canvas[-2:, cw+2:2*cw+2,:] = torch.rot90(img_tensor[2*cw-2:2*cw, 3*cw:,:], 2, [0,1])
    # left
    canvas[cw+2:2*cw+2, 0:2,:] = img_tensor[cw:2*cw, -2:,:]
    # right
    canvas[cw+2:2*cw+2, -2:,:] = img_tensor[cw:2*cw, 0:2,:]

    # Rotate and copy
    canvas[cw:cw+2, :cw+2,:] = torch.rot90(canvas[:cw+2, cw+2:cw+4,:], 1, [0,1])
    canvas[:cw+2, cw:cw+2,:] = torch.rot90(canvas[cw+2:cw+4, :cw+2,:], 3, [0,1])
    canvas[2*cw+2:2*cw+4, :cw+2,:] = torch.rot90(canvas[2*cw+2:, cw+2:cw+4,:], 3, [0,1])
    canvas[2*cw+2:, cw:cw+2,:] = torch.rot90(canvas[2*cw:2*cw+2, :cw+2,:], 1, [0,1])
    canvas[cw:cw+2, 2*cw+2:3*cw+4,:] = torch.rot90(canvas[:cw+2, 2*cw:2*cw+2,:], 3, [0,1])
    canvas[:cw+2, 2*cw+2:2*cw+4,:] = torch.rot90(canvas[cw+2:cw+4, 2*cw+2:3*cw+4,:], 1, [0,1])
    canvas[2*cw+2:2*cw+4, 2*cw+2:3*cw+2,:] = torch.rot90(canvas[2*cw+2:-2, 2*cw:2*cw+2,:], 1, [0,1])
    canvas[2*cw+2:, 2*cw+2:2*cw+4,:] = torch.rot90(canvas[2*cw:2*cw+2, 2*cw+2:3*cw+4,:], 3, [0,1])
    
    # Flip and copy
    #canvas[cw:cw+2, 3*cw+2:,:] = torch.flip(canvas[3:1:-1, 2*cw+1:cw-1:-1,:], [0,1])
    #canvas[2*cw+2:2*cw+4, 3*cw+2:,:] = torch.flip(canvas[-3:-5:-1, 2*cw+1:cw-1:-1,:], [0,1])
    
    # Convert the tensor back to a numpy array
    return canvas.cpu().numpy()

In [36]:
def cube_to_equirectangular_torch(img, width, device):
    # imgをテンソルに変換
    img_tensor = torch.tensor(img, device=device).float()

    img_w = width
    img_h = width // 2
    width = img.shape[1] // 4

    x, y, z = create_3dmap_from_size_torch(img_w, img_h, device)

    w = 0.5

    # front
    xx = w*y / x + w
    yy = w*z / x + w    
    mask = (xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (x > 0)
    tmpx = torch.where(mask, xx*width + width, 0)
    tmpy = torch.where(mask, yy*width + width, 0)
     
    # back
    xx = w*y / x + w
    yy = -w*z / x + w    
    mask = (xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (x < 0)
    tmpx = torch.where(mask, xx*width + width*3, tmpx)
    tmpy = torch.where(mask, yy*width + width, tmpy)
     
    #right
    xx = -w*x / y + w
    yy = w*z / y + w    
    mask = (xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (y > 0)
    tmpx = torch.where(mask, xx*width + width*2, tmpx)
    tmpy = torch.where(mask, yy*width + width, tmpy)
     
    #left
    xx = -w*x / y + w
    yy = -w*z / y + w    
    mask = (xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (y < 0)
    tmpx = torch.where(mask, xx*width, tmpx)
    tmpy = torch.where(mask, yy*width + width, tmpy)
     
    #up
    xx = -w*y / z + w
    yy = -w*x / z + w    
    mask = (xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (z < 0)
    tmpx = torch.where(mask, xx*width + width, tmpx)
    tmpy = torch.where(mask, yy*width, tmpy)
     
    #bottom
    xx = w*y / z + w
    yy = -w*x / z + w    
    mask = (xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (z > 0)
    tmpx = torch.where(mask, xx*width + width, tmpx)
    tmpy = torch.where(mask, yy*width + width*2, tmpy)

    cube = padding_cube(img, device)
    print(cube.shape, type(cube))

    # grid_sampleを使うための座標の変換
    grid = torch.stack((2*y/img_h - 1, 2*x/img_w - 1), dim=-1)
    grid = grid.unsqueeze(0)  # バッチ次元の追加

    # チャンネルの次元を先頭に移動
    #cube = cube.permute(2, 0, 1).unsqueeze(0)
    cube = torch.tensor(cube, device=device).permute(2, 0, 1).float().cpu().numpy().astype(np.float32)

    # grid_sampleを使用してリマップ
    output = cv2.remap(cube, tmpx.cpu().numpy().astype(np.float32), tmpy.cpu().numpy().astype(np.float32), interpolation=cv2.INTER_LINEAR)

    return output

In [37]:
immm = cube_to_equirectangular_torch(cube_img, 1920*2, device)

tensor([[-0.0008, -0.0025, -0.0041,  ..., -0.0025, -0.0008,  0.0008],
        [-0.0008, -0.0025, -0.0041,  ..., -0.0025, -0.0008,  0.0008],
        [-0.0008, -0.0025, -0.0041,  ..., -0.0025, -0.0008,  0.0008],
        ...,
        [-0.0008, -0.0025, -0.0041,  ..., -0.0025, -0.0008,  0.0008],
        [-0.0008, -0.0025, -0.0041,  ..., -0.0025, -0.0008,  0.0008],
        [-0.0008, -0.0025, -0.0041,  ..., -0.0025, -0.0008,  0.0008]],
       device='cuda:0') tensor([[-6.6914e-07, -2.0082e-06, -3.3472e-06,  ..., -2.0088e-06,
         -6.6982e-07,  6.6921e-07],
        [-2.0081e-06, -6.0267e-06, -1.0045e-05,  ..., -6.0287e-06,
         -2.0102e-06,  2.0084e-06],
        [-3.3469e-06, -1.0045e-05, -1.6742e-05,  ..., -1.0048e-05,
         -3.3504e-06,  3.3473e-06],
        ...,
        [ 2.0085e-06,  6.0278e-06,  1.0047e-05,  ...,  6.0299e-06,
          2.0106e-06, -2.0087e-06],
        [ 6.6972e-07,  2.0099e-06,  3.3501e-06,  ...,  2.0106e-06,
          6.7041e-07, -6.6979e-07],
        [-6.69

In [34]:
cv2.imwrite("eq2.jpg", immm)

True

In [21]:
##
# @brief 3次元座標群取得関数
# @details 360度画像の緯度経度が半径1の球の3次元座標のどこにマッピングされるか
# @param img_w 360度画像の幅
# @param img_h 360度画像の高さ
# @return x,y,zの座標群
def create_3dmap_from_size(img_w, img_h):
 
    h = np.linspace(-np.pi/2, np.pi/2, img_h, endpoint=False)
    w = np.linspace(-np.pi, np.pi, img_w, endpoint=False)
     
    # 配列を対称形にするためのオフセット
    h = h + (np.pi/2) / img_h
    w = w + np.pi / img_w
     
    theta, phi = np.meshgrid(w, h)
 
    x = np.cos(phi) * np.cos(theta)
    y = np.cos(phi) * np.sin(theta)
    z = np.sin(phi)
 
    return x, y, z
 
 
##
# @brief cube画像にのりしろをつける
# @details 画素補間用にcube画像の境界部に2画素分反対側の画像を付与する
# @param img cube画像
# @return のりしろ付き画像
def add_norishiro_to_cube(img):
     
    h,w,c = img.shape
    cw = w // 4
     
    # 周囲2画素大きい画像を用意
    canvas = np.zeros((h+4, w+4, c), dtype=img.dtype)
    canvas[2:-2, 2:-2,:] = img
     
    # 上下左右にのりしろをつける
    # up    
    canvas[0:2,cw+2:2*cw+2,:] = np.rot90(img[cw:cw+2, 3*cw:,:], 2)
    # bottom
    canvas[-2:,cw+2:2*cw+2,:] = np.rot90(img[2*cw-2:2*cw,3*cw:,:], 2)
    # left
    canvas[cw+2:2*cw+2,0:2,:] = img[cw:2*cw,-2:,:]
    # right
    canvas[cw+2:2*cw+2,-2:,:] = img[cw:2*cw,0:2,:]
 
    #　残りの折り返しの部分のコピー
    canvas[cw:cw+2,:cw+2,:] = np.rot90(canvas[:cw+2,cw+2:cw+4,:])
    canvas[:cw+2,cw:cw+2,:] = np.rot90(canvas[cw+2:cw+4,:cw+2,:],3)
    #
    canvas[2*cw+2:2*cw+4,:cw+2,:] = np.rot90(canvas[2*cw+2:,cw+2:cw+4,:],3)
    canvas[2*cw+2:,cw:cw+2,:] = np.rot90(canvas[2*cw:2*cw+2,:cw+2,:])
    #
    canvas[cw:cw+2,2*cw+2:3*cw+2,:] = np.rot90(canvas[2:cw+2,2*cw:2*cw+2,:],3)
    canvas[:cw+2,2*cw+2:2*cw+4:] = np.rot90(canvas[cw+2:cw+4,2*cw+2:3*cw+4,:])
    #
    canvas[2*cw+2:2*cw+4,2*cw+2:3*cw+2,:] = np.rot90(canvas[2*cw+2:-2,2*cw:2*cw+2,:])
    canvas[2*cw+2:,2*cw+2:2*cw+4,:] = np.rot90(canvas[2*cw:2*cw+2,2*cw+2:3*cw+4,:], 3)
 
    #
    canvas[cw:cw+2, 3*cw+2:,:] = canvas[3:1:-1, 2*cw+1:cw-1:-1,:]
    canvas[2*cw+2:2*cw+4, 3*cw+2:,:] = canvas[-3:-5:-1, 2*cw+1:cw-1:-1,:]
     
    return canvas
 
 
##
# @brief キューブ画像から全天球画像を作る
# @details 6面を展開図から全天球画像へ変換する
# @param img 元となるキューブ（展開図）画像
# @param width 全天球画像の幅(2の倍数である必要がある)
# @return 変換された画像
def cube_to_equirectangular(img, width, interpolation=cv2.INTER_LINEAR):
 
    img_w = width
    img_h = width // 2
    width = img.shape[1] // 4
     
    x, y, z = create_3dmap_from_size(img_w, img_h)
    w = 0.5
     
    # front
    xx = w*y / x + w
    yy = w*z / x + w    
    mask = np.where((xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (x > 0), 1, 0)
    tmpx = np.where(mask, xx*width + width, 0)
    tmpy = np.where(mask, yy*width + width, 0)
     
    # back
    xx = w*y / x + w
    yy = -w*z / x + w    
    mask = np.where((xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (x < 0), 1, 0)
    tmpx = np.where(mask, xx*width + width*3, tmpx)
    tmpy = np.where(mask, yy*width + width, tmpy)
     
    #right
    xx = -w*x / y + w
    yy = w*z / y + w    
    mask = np.where((xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (y > 0), 1, 0)
    tmpx = np.where(mask, xx*width + width*2, tmpx)
    tmpy = np.where(mask, yy*width + width, tmpy)
     
    #left
    xx = -w*x / y + w
    yy = -w*z / y + w    
    mask = np.where((xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (y < 0), 1, 0)
    tmpx = np.where(mask, xx*width, tmpx)
    tmpy = np.where(mask, yy*width + width, tmpy)
     
    #up
    xx = -w*y / z + w
    yy = -w*x / z + w    
    mask = np.where((xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (z < 0), 1, 0)
    tmpx = np.where(mask, xx*width + width, tmpx)
    tmpy = np.where(mask, yy*width, tmpy)
     
    #bottom
    xx = w*y / z + w
    yy = -w*x / z + w    
    mask = np.where((xx > 0) & (xx < 1) & (yy > 0) & (yy < 1) & (z > 0), 1, 0)
    tmpx = np.where(mask, xx*width + width, tmpx)
    tmpy = np.where(mask, yy*width + width*2, tmpy)
         
    cube = add_norishiro_to_cube(img)
     
    # のりしろ2画素と座標を画素位置に戻すため0.5オフセット
    tmpx += (2.0 - 0.5)
    tmpy += (2.0 - 0.5)
     
    return cv2.remap(cube, tmpx.astype(np.float32), tmpy.astype(np.float32), interpolation)

In [22]:
out = cube_to_equirectangular(cube_img, 1920*2, interpolation=cv2.INTER_CUBIC)
cv2.imwrite("eq.jpg",out)

[[-0.00081812 -0.00081812 -0.00081812 ... -0.00081812 -0.00081812
  -0.00081812]
 [-0.00245437 -0.00245436 -0.00245435 ... -0.00245435 -0.00245436
  -0.00245437]
 [-0.0040906  -0.00409059 -0.00409057 ... -0.00409057 -0.00409059
  -0.0040906 ]
 ...
 [-0.0040906  -0.00409059 -0.00409057 ... -0.00409057 -0.00409059
  -0.0040906 ]
 [-0.00245437 -0.00245436 -0.00245435 ... -0.00245435 -0.00245436
  -0.00245437]
 [-0.00081812 -0.00081812 -0.00081812 ... -0.00081812 -0.00081812
  -0.00081812]] [[-6.69325236e-07 -2.00797392e-06 -3.34661722e-06 ...  3.34661722e-06
   2.00797392e-06  6.69325236e-07]
 [-2.00797392e-06 -6.02391637e-06 -1.00398427e-05 ...  1.00398427e-05
   6.02391637e-06  2.00797392e-06]
 [-3.34661722e-06 -1.00398427e-05 -1.67330413e-05 ...  1.67330413e-05
   1.00398427e-05  3.34661722e-06]
 ...
 [-3.34661722e-06 -1.00398427e-05 -1.67330413e-05 ...  1.67330413e-05
   1.00398427e-05  3.34661722e-06]
 [-2.00797392e-06 -6.02391637e-06 -1.00398427e-05 ...  1.00398427e-05
   6.02391637

True